In [342]:
!pip install pyspark

In [343]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.functions import mean,col,split, col, regexp_extract, when, lit
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import QuantileDiscretizer

In [344]:
spark = SparkSession \
    .builder \
    .appName("Spark ML for titanic data ") \
    .getOrCreate()

In [345]:
df_train = spark.read.csv('../input/titanic/train.csv', header = True, inferSchema=True)
df_test = spark.read.csv('../input/titanic/train.csv', header = True, inferSchema=True)

In [346]:
titanic_train = df_train.alias("titanic_train")

In [347]:
df_train.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [348]:
df_train.select("Survived","Pclass","Embarked").show()

+--------+------+--------+
|Survived|Pclass|Embarked|
+--------+------+--------+
|       0|     3|       S|
|       1|     1|       C|
|       1|     3|       S|
|       1|     1|       S|
|       0|     3|       S|
|       0|     3|       Q|
|       0|     1|       S|
|       0|     3|       S|
|       1|     3|       S|
|       1|     2|       C|
|       1|     3|       S|
|       1|     1|       S|
|       0|     3|       S|
|       0|     3|       S|
|       0|     3|       S|
|       1|     2|       S|
|       0|     3|       Q|
|       1|     2|       S|
|       0|     3|       S|
|       1|     3|       C|
+--------+------+--------+
only showing top 20 rows



In [349]:
df_train.groupBy("Survived").count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



In [350]:
grpby_output = df_train.groupBy("Survived").count()

In [351]:
grpby_output.show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



In [352]:
df_train.groupBy("Sex","Survived").count().show()

+------+--------+-----+
|   Sex|Survived|count|
+------+--------+-----+
|  male|       0|  468|
|female|       1|  233|
|female|       0|   81|
|  male|       1|  109|
+------+--------+-----+



In [353]:
# combined_dt = df_train.join(df_test,['PassengerId'],how='inner')
combined_dt = df_train.union(df_test)
combined_dt.show()
#df_test.printSchema()
#df_test.collect()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [354]:
#combined_dt.write.csv('../titanic/output.csv')

In [355]:
# Checking null values and null count 
def null_value_count(df):
  null_columns_counts = []
  numRows = df.count()
  for k in df.columns:
    nullRows = df.where(col(k).isNull()).count()
    if(nullRows > 0):
      temp = k,nullRows
      null_columns_counts.append(temp)
  return(null_columns_counts)

In [356]:
null_cols_counts = null_value_count(combined_dt)

In [357]:
combined_dt.where(combined_dt['Age'].isNull()).show()

+-----------+--------+------+--------------------+------+----+-----+-----+---------------+--------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|         Ticket|    Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+---------------+--------+-----+--------+
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|         330877|  8.4583| null|       Q|
|         18|       1|     2|Williams, Mr. Cha...|  male|null|    0|    0|         244373|    13.0| null|       S|
|         20|       1|     3|Masselmani, Mrs. ...|female|null|    0|    0|           2649|   7.225| null|       C|
|         27|       0|     3|Emir, Mr. Farred ...|  male|null|    0|    0|           2631|   7.225| null|       C|
|         29|       1|     3|"O'Dwyer, Miss. E...|female|null|    0|    0|         330959|  7.8792| null|       Q|
|         30|       0|     3| Todoroff, Mr. Lalio|  male|null|    0|    0|      

In [358]:
combined_dt.where(combined_dt['Fare'].isNull()).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [359]:
combined_dt.where(combined_dt['Age'].isNull()).show()

+-----------+--------+------+--------------------+------+----+-----+-----+---------------+--------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|         Ticket|    Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+---------------+--------+-----+--------+
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|         330877|  8.4583| null|       Q|
|         18|       1|     2|Williams, Mr. Cha...|  male|null|    0|    0|         244373|    13.0| null|       S|
|         20|       1|     3|Masselmani, Mrs. ...|female|null|    0|    0|           2649|   7.225| null|       C|
|         27|       0|     3|Emir, Mr. Farred ...|  male|null|    0|    0|           2631|   7.225| null|       C|
|         29|       1|     3|"O'Dwyer, Miss. E...|female|null|    0|    0|         330959|  7.8792| null|       Q|
|         30|       0|     3| Todoroff, Mr. Lalio|  male|null|    0|    0|      

In [360]:
combined_dt.where(combined_dt['Fare'].isNull()).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [361]:
combined_dt.where(combined_dt['Cabin'].isNull()).show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|          330877| 8.4583| null|       Q|
|          8|       0|     3|Palsson, Master. ...|  male| 2.0|    3|    1|          349909| 21.075| null|       S|
|          9|       1|     3|Johnson, Mrs. Osc...|female|27.0|    0|    2|      

In [362]:
combined_dt = combined_dt.na.fill('N', subset=['Cabin'])
#df_test = df_test.na.fill('N', subset=['Cabin'])

In [363]:
missing_value = combined_dt.filter(
    (combined_dt['Pclass'] == 3) &
    (combined_dt.Embarked == 'S') &
    (combined_dt.Sex == "male")
)
## filling in the null value in the fare column using Fare mean. 
combined_dt = combined_dt.na.fill(
    missing_value.select(mean('Fare')).collect()[0][0],
    subset=['Fare']
)

In [364]:
combined_dt.where(combined_dt['Fare'].isNull()).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [365]:
combined_dt = combined_dt.na.fill('C', subset=['Embarked'])

In [366]:
combined_dt.where(combined_dt['Embarked'].isNull()).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [367]:
#df_test.where(df_test.Embarked.isNull()).show()

In [368]:
combined_dt.where(combined_dt['Cabin'].isNull()).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [369]:
combined_dt.where(combined_dt['Cabin'].isNull()).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [370]:
combined_dt.select("Name").count()

1782

In [371]:
combined_dt.select("Name").show()

+--------------------+
|                Name|
+--------------------+
|Braund, Mr. Owen ...|
|Cumings, Mrs. Joh...|
|Heikkinen, Miss. ...|
|Futrelle, Mrs. Ja...|
|Allen, Mr. Willia...|
|    Moran, Mr. James|
|McCarthy, Mr. Tim...|
|Palsson, Master. ...|
|Johnson, Mrs. Osc...|
|Nasser, Mrs. Nich...|
|Sandstrom, Miss. ...|
|Bonnell, Miss. El...|
|Saundercock, Mr. ...|
|Andersson, Mr. An...|
|Vestrom, Miss. Hu...|
|Hewlett, Mrs. (Ma...|
|Rice, Master. Eugene|
|Williams, Mr. Cha...|
|Vander Planke, Mr...|
|Masselmani, Mrs. ...|
+--------------------+
only showing top 20 rows



In [372]:
combined_dt.filter(combined_dt.Age==46).select("Name").show()

+--------------------+
|                Name|
+--------------------+
|Chaffee, Mr. Herb...|
|McKane, Mr. Peter...|
|Guggenheim, Mr. B...|
|Chaffee, Mr. Herb...|
|McKane, Mr. Peter...|
|Guggenheim, Mr. B...|
+--------------------+



In [373]:
combined_dt.select("Age").show()

+----+
| Age|
+----+
|22.0|
|38.0|
|26.0|
|35.0|
|35.0|
|null|
|54.0|
| 2.0|
|27.0|
|14.0|
| 4.0|
|58.0|
|20.0|
|39.0|
|14.0|
|55.0|
| 2.0|
|null|
|31.0|
|null|
+----+
only showing top 20 rows



In [374]:
#Adding family size
combined_dt = combined_dt.withColumn("Family_Size",col('SibSp')+col('Parch'))

In [375]:
combined_dt.groupBy("Family_Size").count().show()

+-----------+-----+
|Family_Size|count|
+-----------+-----+
|          1|  322|
|          6|   24|
|          3|   58|
|          5|   44|
|          4|   30|
|          7|   12|
|         10|   14|
|          2|  204|
|          0| 1074|
+-----------+-----+



In [376]:
#Adding alone
combined_dt = combined_dt.withColumn('Alone',lit(0))

In [377]:
combined_dt = combined_dt.withColumn("Alone",when(combined_dt["Family_Size"] == 0, 1).otherwise(combined_dt["Alone"]))

In [378]:
combined_dt.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked',
 'Family_Size',
 'Alone']

In [379]:
combined_dt.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----------+-----+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Family_Size|Alone|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----------+-----+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|    N|       S|          1|    0|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|          1|    0|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925|    N|       S|          0|    1|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|          1|    0|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0| 

In [380]:
combined_dt.where(combined_dt['Age'].isNull()).show()

+-----------+--------+------+--------------------+------+----+-----+-----+---------------+--------+-----+--------+-----------+-----+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|         Ticket|    Fare|Cabin|Embarked|Family_Size|Alone|
+-----------+--------+------+--------------------+------+----+-----+-----+---------------+--------+-----+--------+-----------+-----+
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|         330877|  8.4583|    N|       Q|          0|    1|
|         18|       1|     2|Williams, Mr. Cha...|  male|null|    0|    0|         244373|    13.0|    N|       S|          0|    1|
|         20|       1|     3|Masselmani, Mrs. ...|female|null|    0|    0|           2649|   7.225|    N|       C|          0|    1|
|         27|       0|     3|Emir, Mr. Farred ...|  male|null|    0|    0|           2631|   7.225|    N|       C|          0|    1|
|         29|       1|     3|"O'Dwyer, Miss. E...|female|null|    0| 

In [381]:
combined_dt.where(combined_dt['Cabin'].isNull()).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----------+-----+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|Family_Size|Alone|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----------+-----+
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----------+-----+



In [382]:
combined_dt = combined_dt.withColumn("Initial",regexp_extract(col("Name"),"([A-Za-z]+)\.",1))

In [383]:
combined_dt.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----------+-----+-------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Family_Size|Alone|Initial|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----------+-----+-------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|    N|       S|          1|    0|     Mr|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|          1|    0|    Mrs|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925|    N|       S|          0|    1|   Miss|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|          1|    0|    Mrs|
|          5|

In [384]:
combined_dt.select("Initial").distinct().show()


+--------+
| Initial|
+--------+
|     Don|
|    Miss|
|Countess|
|     Col|
|     Rev|
|    Lady|
|  Master|
|     Mme|
|    Capt|
|      Mr|
|      Dr|
|     Mrs|
|     Sir|
|Jonkheer|
|    Mlle|
|   Major|
|      Ms|
+--------+



In [385]:
combined_dt = combined_dt.replace(['Mlle','Mme', 'Ms', 'Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
               ['Miss','Miss','Miss','Mr','Mr',  'Mrs',  'Mrs',  'Other',  'Other','Other','Mr','Mr','Mr'])

In [386]:
combined_dt.select("Initial").distinct().show()

+-------+
|Initial|
+-------+
|   Miss|
|  Other|
| Master|
|     Mr|
|    Mrs|
+-------+



In [387]:
combined_dt.groupby('Initial').avg('Age').collect()

[Row(Initial='Miss', avg(Age)=21.86),
 Row(Initial='Other', avg(Age)=45.888888888888886),
 Row(Initial='Master', avg(Age)=4.574166666666667),
 Row(Initial='Mr', avg(Age)=32.73960880195599),
 Row(Initial='Mrs', avg(Age)=35.981818181818184)]

In [388]:
combined_dt = combined_dt.withColumn("Age",when((combined_dt["Initial"] == "Miss") & (combined_dt["Age"].isNull()), 22).otherwise(combined_dt["Age"]))
combined_dt = combined_dt.withColumn("Age",when((combined_dt["Initial"] == "Other") & (combined_dt["Age"].isNull()), 46).otherwise(combined_dt["Age"]))
combined_dt = combined_dt.withColumn("Age",when((combined_dt["Initial"] == "Master") & (combined_dt["Age"].isNull()), 5).otherwise(combined_dt["Age"]))
combined_dt = combined_dt.withColumn("Age",when((combined_dt["Initial"] == "Mr") & (combined_dt["Age"].isNull()), 33).otherwise(combined_dt["Age"]))
combined_dt = combined_dt.withColumn("Age",when((combined_dt["Initial"] == "Mrs") & (combined_dt["Age"].isNull()), 36).otherwise(combined_dt["Age"]))

In [389]:
combined_dt.select("Age").show()

+----+
| Age|
+----+
|22.0|
|38.0|
|26.0|
|35.0|
|35.0|
|33.0|
|54.0|
| 2.0|
|27.0|
|14.0|
| 4.0|
|58.0|
|20.0|
|39.0|
|14.0|
|55.0|
| 2.0|
|33.0|
|31.0|
|36.0|
+----+
only showing top 20 rows



In [390]:
combined_dt.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = false)
 |-- Cabin: string (nullable = false)
 |-- Embarked: string (nullable = false)
 |-- Family_Size: integer (nullable = true)
 |-- Alone: integer (nullable = false)
 |-- Initial: string (nullable = true)



In [391]:
combined_dt.where(combined_dt['Embarked'].isNull()).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----------+-----+-------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|Family_Size|Alone|Initial|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----------+-----+-------+
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----------+-----+-------+



In [392]:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(combined_dt) for column in ["Sex","Embarked","Initial"]]
pipeline = Pipeline(stages=indexers)
combined_dt = pipeline.fit(combined_dt).transform(combined_dt)

In [393]:
combined_dt.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----------+-----+-------+---------+--------------+-------------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Family_Size|Alone|Initial|Sex_index|Embarked_index|Initial_index|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----------+-----+-------+---------+--------------+-------------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|    N|       S|          1|    0|     Mr|      0.0|           0.0|          0.0|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|          1|    0|    Mrs|      1.0|           1.0|          2.0|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| 

In [394]:
combined_dt.printSchema()


root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = false)
 |-- Cabin: string (nullable = false)
 |-- Embarked: string (nullable = false)
 |-- Family_Size: integer (nullable = true)
 |-- Alone: integer (nullable = false)
 |-- Initial: string (nullable = true)
 |-- Sex_index: double (nullable = false)
 |-- Embarked_index: double (nullable = false)
 |-- Initial_index: double (nullable = false)



In [395]:
combined_dt = combined_dt.drop("PassengerId","Name","Ticket","Cabin","Embarked","Sex","Initial")

In [396]:
combined_pd = combined_dt.toPandas()

In [397]:
combined_pd

,Survived,Pclass,Age,SibSp,Parch,Fare,Family_Size,Alone,Sex_index,Embarked_index,Initial_index
0,0,3,22.0,1,0,7.2500,1,0,0.0,0.0,0.0
1,1,1,38.0,1,0,71.2833,1,0,1.0,1.0,2.0
2,1,3,26.0,0,0,7.9250,0,1,1.0,0.0,1.0
3,1,1,35.0,1,0,53.1000,1,0,1.0,0.0,2.0
4,0,3,35.0,0,0,8.0500,0,1,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1777,0,2,27.0,0,0,13.0000,0,1,0.0,0.0,4.0
1778,1,1,19.0,0,0,30.0000,0,1,1.0,0.0,1.0
1779,0,3,22.0,1,2,23.4500,3,0,1.0,0.0,1.0
1780,1,1,26.0,0,0,30.0000,0,1,0.0,1.0,0.0


In [398]:
feature = VectorAssembler(inputCols=combined_dt.columns[1:],outputCol="features")
feature_vector= feature.transform(combined_dt)

In [399]:
feature_vector.show()

+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+--------------------+
|Survived|Pclass| Age|SibSp|Parch|   Fare|Family_Size|Alone|Sex_index|Embarked_index|Initial_index|            features|
+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+--------------------+
|       0|     3|22.0|    1|    0|   7.25|          1|    0|      0.0|           0.0|          0.0|(10,[0,1,2,4,5],[...|
|       1|     1|38.0|    1|    0|71.2833|          1|    0|      1.0|           1.0|          2.0|[1.0,38.0,1.0,0.0...|
|       1|     3|26.0|    0|    0|  7.925|          0|    1|      1.0|           0.0|          1.0|[3.0,26.0,0.0,0.0...|
|       1|     1|35.0|    1|    0|   53.1|          1|    0|      1.0|           0.0|          2.0|[1.0,35.0,1.0,0.0...|
|       0|     3|35.0|    0|    0|   8.05|          0|    1|      0.0|           0.0|          0.0|(10,[0,1,4,6],[3....|
|       0|     3|33.0|    0|    

In [400]:
train_pd = combined_pd[:df_train.count()]
test_pd = combined_pd[df_train.count():]

In [401]:
train_pd

,Survived,Pclass,Age,SibSp,Parch,Fare,Family_Size,Alone,Sex_index,Embarked_index,Initial_index
0,0,3,22.0,1,0,7.2500,1,0,0.0,0.0,0.0
1,1,1,38.0,1,0,71.2833,1,0,1.0,1.0,2.0
2,1,3,26.0,0,0,7.9250,0,1,1.0,0.0,1.0
3,1,1,35.0,1,0,53.1000,1,0,1.0,0.0,2.0
4,0,3,35.0,0,0,8.0500,0,1,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,27.0,0,0,13.0000,0,1,0.0,0.0,4.0
887,1,1,19.0,0,0,30.0000,0,1,1.0,0.0,1.0
888,0,3,22.0,1,2,23.4500,3,0,1.0,0.0,1.0
889,1,1,26.0,0,0,30.0000,0,1,0.0,1.0,0.0


In [402]:
df_train = spark.createDataFrame(train_pd)
df_test = spark.createDataFrame(test_pd)
df_test = df_test.drop('Survived')
#df_test.show()


In [403]:
assembler = VectorAssembler(inputCols=df_train.columns[1:],outputCol="features")
train_assembler_vector = assembler.transform(df_train)
train_assembler_vector.show()

+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+--------------------+
|Survived|Pclass| Age|SibSp|Parch|   Fare|Family_Size|Alone|Sex_index|Embarked_index|Initial_index|            features|
+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+--------------------+
|       0|     3|22.0|    1|    0|   7.25|          1|    0|      0.0|           0.0|          0.0|(10,[0,1,2,4,5],[...|
|       1|     1|38.0|    1|    0|71.2833|          1|    0|      1.0|           1.0|          2.0|[1.0,38.0,1.0,0.0...|
|       1|     3|26.0|    0|    0|  7.925|          0|    1|      1.0|           0.0|          1.0|[3.0,26.0,0.0,0.0...|
|       1|     1|35.0|    1|    0|   53.1|          1|    0|      1.0|           0.0|          2.0|[1.0,35.0,1.0,0.0...|
|       0|     3|35.0|    0|    0|   8.05|          0|    1|      0.0|           0.0|          0.0|(10,[0,1,4,6],[3....|
|       0|     3|33.0|    0|    

In [404]:
test_assembler = VectorAssembler(inputCols=df_test.columns,outputCol="features")
test_assembler_vector = test_assembler.transform(df_test)
test_assembler_vector.show()

+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+--------------------+
|Pclass| Age|SibSp|Parch|   Fare|Family_Size|Alone|Sex_index|Embarked_index|Initial_index|            features|
+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+--------------------+
|     3|22.0|    1|    0|   7.25|          1|    0|      0.0|           0.0|          0.0|(10,[0,1,2,4,5],[...|
|     1|38.0|    1|    0|71.2833|          1|    0|      1.0|           1.0|          2.0|[1.0,38.0,1.0,0.0...|
|     3|26.0|    0|    0|  7.925|          0|    1|      1.0|           0.0|          1.0|[3.0,26.0,0.0,0.0...|
|     1|35.0|    1|    0|   53.1|          1|    0|      1.0|           0.0|          2.0|[1.0,35.0,1.0,0.0...|
|     3|35.0|    0|    0|   8.05|          0|    1|      0.0|           0.0|          0.0|(10,[0,1,4,6],[3....|
|     3|33.0|    0|    0| 8.4583|          0|    1|      0.0|           2.0|          0.0|(10,[0,1,4,6,8

In [405]:
(trainData, testData) = train_assembler_vector.randomSplit([0.8, 0.2],seed = 11)

In [406]:
from pyspark.ml.classification import LinearSVC
svm = LinearSVC(labelCol="Survived", featuresCol="features")
svm_model = svm.fit(trainData)
svm_prediction = svm_model.transform(testData)
svm_prediction.select("prediction", "Survived", "features").show()

+----------+--------+--------------------+
|prediction|Survived|            features|
+----------+--------+--------------------+
|       0.0|       0|[1.0,28.0,1.0,0.0...|
|       0.0|       0|(10,[0,1,4,6],[1....|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|(10,[0,1,4,6],[1....|
|       0.0|       0|[1.0,51.0,0.0,1.0...|
|       0.0|       0|(10,[0,1,4,6],[1....|
|       0.0|       0|[1.0,65.0,0.0,1.0...|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       1.0|       0|[2.0,24.0,0.0,0.0...|
|       1.0|       0|[2.0,27.0,1.0,0.0...|
|       0.0|       0|(10,[0,1,4,6],[2....|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|[3.0,1.0,4.0,1.0,...|
|       0.0|       0|[3.0,2.0,4.0,2.0,...|
|       1.0|       0|[3.0,17.0,0.0,0.0...|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|(10,[0,1,4,6],[3....|
|       0.0|       0|(10,[0,1,4,6],[3....|
|       0.0|       0|(10,[0,1,4,6],[3....|
+----------

In [407]:
svm_prediction.filter(df_train.Survived==1).show()

+--------+------+----+-----+-----+--------+-----------+-----+---------+--------------+-------------+--------------------+--------------------+----------+
|Survived|Pclass| Age|SibSp|Parch|    Fare|Family_Size|Alone|Sex_index|Embarked_index|Initial_index|            features|       rawPrediction|prediction|
+--------+------+----+-----+-----+--------+-----------+-----+---------+--------------+-------------+--------------------+--------------------+----------+
|       1|     1|28.0|    0|    0|    35.5|          0|    1|      0.0|           0.0|          0.0|(10,[0,1,4,6],[1....|[0.71586811855944...|       0.0|
|       1|     1|36.0|    0|    1|    55.0|          1|    0|      1.0|           0.0|          2.0|[1.0,36.0,0.0,1.0...|[-1.8292416941613...|       1.0|
|       1|     1|36.0|    1|    0|146.5208|          1|    0|      1.0|           1.0|          2.0|[1.0,36.0,1.0,0.0...|[-2.0453268572373...|       1.0|
|       1|     1|40.0|    0|    0|    31.0|          0|    1|      0.0|     

In [408]:
svm_prediction.filter(df_train.Survived==1).count()

72

In [409]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="Survived", predictionCol="prediction", metricName="accuracy")
svm_accuracy = evaluator.evaluate(svm_prediction)
print("Accuracy of Support Vector Machine is = %g"% (svm_accuracy))
print("Test Error of Support Vector Machine = %g " % (1.0 - svm_accuracy))

Accuracy of Support Vector Machine is = 0.802198
Test Error of Support Vector Machine = 0.197802 


In [410]:
submission = spark.read.csv('../input/titanic/gender_submission.csv', header = True, inferSchema=True)
submission.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)



In [411]:
df_test = df_test.drop('Survived')

In [412]:
f_predictions = svm_model.transform(test_assembler_vector)
f_predictions = f_predictions.toPandas()

In [413]:
f_predictions

,Pclass,Age,SibSp,Parch,Fare,Family_Size,Alone,Sex_index,Embarked_index,Initial_index,features,rawPrediction,prediction
0,3,22.0,1,0,7.2500,1,0,0.0,0.0,0.0,"(3.0, 22.0, 1.0, 0.0, 7.25, 1.0, 0.0, 0.0, 0.0...","[1.1184091285261863, -1.1184091285261863]",0.0
1,1,38.0,1,0,71.2833,1,0,1.0,1.0,2.0,"[1.0, 38.0, 1.0, 0.0, 71.2833, 1.0, 0.0, 1.0, ...","[-1.7392219229348194, 1.7392219229348194]",1.0
2,3,26.0,0,0,7.9250,0,1,1.0,0.0,1.0,"[3.0, 26.0, 0.0, 0.0, 7.925, 0.0, 1.0, 1.0, 0....","[-0.8006572085123602, 0.8006572085123602]",1.0
3,1,35.0,1,0,53.1000,1,0,1.0,0.0,2.0,"[1.0, 35.0, 1.0, 0.0, 53.1, 1.0, 0.0, 1.0, 0.0...","[-1.6474216995699822, 1.6474216995699822]",1.0
4,3,35.0,0,0,8.0500,0,1,0.0,0.0,0.0,"(3.0, 35.0, 0.0, 0.0, 8.05, 0.0, 1.0, 0.0, 0.0...","[1.2651247040557525, -1.2651247040557525]",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,2,27.0,0,0,13.0000,0,1,0.0,0.0,4.0,"(2.0, 27.0, 0.0, 0.0, 13.0, 0.0, 1.0, 0.0, 0.0...","[-1.1088984889716544, 1.1088984889716544]",1.0
887,1,19.0,0,0,30.0000,0,1,1.0,0.0,1.0,"[1.0, 19.0, 0.0, 0.0, 30.0, 0.0, 1.0, 1.0, 0.0...","[-1.3301804204542163, 1.3301804204542163]",1.0
888,3,22.0,1,2,23.4500,3,0,1.0,0.0,1.0,"[3.0, 22.0, 1.0, 2.0, 23.45, 3.0, 0.0, 1.0, 0....","[-0.22013031289694623, 0.22013031289694623]",1.0
889,1,26.0,0,0,30.0000,0,1,0.0,1.0,0.0,"(1.0, 26.0, 0.0, 0.0, 30.0, 0.0, 1.0, 0.0, 1.0...","[0.636308003700631, -0.636308003700631]",0.0


In [414]:
submission = submission.toPandas()
submission['Survived'] = f_predictions['prediction']
submission

,PassengerId,Survived
0,892,0.0
1,893,1.0
2,894,1.0
3,895,1.0
4,896,0.0
...,...,...
413,1305,0.0
414,1306,0.0
415,1307,1.0
416,1308,1.0


In [415]:
submission['Survived'] = submission['Survived'].astype(int)

In [416]:
submission.to_csv("submission.csv",index=False)